In [ ]:
import cv2
import numpy as np

def extract_features(contour):
    x, y, w, h = cv2.boundingRect(contour)

    aspect_ratio = float(w) / h

    area = cv2.contourArea(contour)
    rect_area = w * h
    extent = float(area) / rect_area if rect_area > 0 else 0

    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area if hull_area > 0 else 0

    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments).flatten()

    return aspect_ratio, extent, solidity, circularity, hu_moments

image_path = r"C:\Users\Admin\Desktop\images (9).jpeg"
image = cv2.imread(image_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (5, 5), 0)

threshold = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

screw_count = 0
bolt_count = 0
min_contour_area = 300

for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_contour_area:
        x, y, w, h = cv2.boundingRect(contour)

        aspect_ratio, extent, solidity, circularity, hu_moments = extract_features(contour)

        if circularity < 0.5 and aspect_ratio > 1.2 and solidity < 0.9:
            screw_count += 1
            label = "Screw"
            color = (0, 255, 0)
        else:
            bolt_count += 1
            label = "Bolt"
            color = (255, 0, 0)

        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

print(f"Number of screws detected: {screw_count}")
print(f"Number of bolts detected: {bolt_count}")

image = cv2.resize(image, (700, 700))
cv2.imshow("Detected Screws and Bolts", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
